In [1]:
from sklearn.feature_extraction.text import CountVectorizer
X = ("Computers can analyze text",
 "They do it using vectors and matrices",
 "Computers can process massive amounts of text data")
vectorizer=CountVectorizer(stop_words='english')
X_vectorized=vectorizer.fit_transform(X)

In [2]:
print(vectorizer.vocabulary_)


{'computers': 2, 'analyze': 1, 'text': 7, 'using': 8, 'vectors': 9, 'matrices': 5, 'process': 6, 'massive': 4, 'amounts': 0, 'data': 3}


In [3]:
print(X_vectorized.todense())

[[0 1 1 0 0 0 0 1 0 0]
 [0 0 0 0 0 1 0 0 1 1]
 [1 0 1 1 1 0 1 1 0 0]]


Exploring the bag of words architecture

In [4]:
import nltk
nltk.download('stopwords')
nltk.download('wordnet')
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer
from nltk.stem.snowball import SnowballStemmer
from nltk.stem.wordnet import WordNetLemmatizer
import pandas as pd
import re
import numpy as np


[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...


In [5]:
sentences = ["We are reading about Natural Language Processing Here",
"Natural Language Processing making computers comprehend language data",
"The field of Natural Language Processing is evolving everyday"]

In [6]:
corpus=pd.Series(sentences)
corpus


0    We are reading about Natural Language Processi...
1    Natural Language Processing making computers c...
2    The field of Natural Language Processing is ev...
dtype: object

In [7]:
def text_clean(corpus, keep_list):
    '''
    Purpose : Function to keep only alphabets, digits and certain words (punctuations, qmarks, tabs etc. removed)

    Input : Takes a text corpus, 'corpus' to be cleaned along with a list of words, 'keep_list', which have to be retained
            even after the cleaning process

    Output : Returns the cleaned text corpus

    '''
    cleaned_corpus = pd.Series()
    for row in corpus:
        qs = []
        for word in row.split():
            if word not in keep_list:
                p1 = re.sub(pattern='[^a-zA-Z0-9]',repl=' ',string=word)
                p1 = p1.lower()
                qs.append(p1)
            else : qs.append(word)
        cleaned_corpus = cleaned_corpus.append(pd.Series(' '.join(qs)))
    return cleaned_corpus

In [8]:
def stopwords_removal(corpus):
    wh_words = ['who', 'what', 'when', 'why', 'how', 'which', 'where', 'whom']
    stop = set(stopwords.words('english'))
    for word in wh_words:
        stop.remove(word)
    corpus = [[x for x in x.split() if x not in stop] for x in corpus]
    return corpus

In [9]:
def lemmatize(corpus):
    lem = WordNetLemmatizer()
    corpus = [[lem.lemmatize(x, pos = 'v') for x in x] for x in corpus]
    return corpus

In [10]:
def stem(corpus, stem_type = None):
    if stem_type=='snowball':
      stemmer=SnowballStemmer(lanaguage='english')
      corpus=[[stemmer.stem(x) for x in x ] for x in corpus]
    else:
      stemmer=PorterStemmer()
      corpus=[[stemmer.stem(x) for x in x] for x in corpus]


In [11]:
def preprocess(corpus, keep_list, cleaning = True, stemming = False, stem_type = None, lemmatization = False, remove_stopwords = True):
    '''
    Purpose : Function to perform all pre-processing tasks (cleaning, stemming, lemmatization, stopwords removal etc.)

    Input :
    'corpus' - Text corpus on which pre-processing tasks will be performed
    'keep_list' - List of words to be retained during cleaning process
    'cleaning', 'stemming', 'lemmatization', 'remove_stopwords' - Boolean variables indicating whether a particular task should
                                                                  be performed or not
    'stem_type' - Choose between Porter stemmer or Snowball(Porter2) stemmer. Default is "None", which corresponds to Porter
                  Stemmer. 'snowball' corresponds to Snowball Stemmer

    Note : Either stemming or lemmatization should be used. There's no benefit of using both of them together

    Output : Returns the processed text corpus

    '''

    if cleaning == True:
        corpus = text_clean(corpus, keep_list)

    if remove_stopwords == True:
        corpus = stopwords_removal(corpus)
    else :
        corpus = [[x for x in x.split()] for x in corpus]

    if lemmatization == True:
        corpus = lemmatize(corpus)


    if stemming == True:
        corpus = stem(corpus, stem_type)

    corpus = [' '.join(x) for x in corpus]

    return corpus

In [12]:
common_dot_words = ['U.S.', 'Mr.', 'Mrs.', 'D.C.']

In [13]:
# Preprocessing with Lemmatization here
preprocessed_corpus = preprocess(corpus, keep_list = common_dot_words, stemming = False, stem_type = None,
                                lemmatization = True, remove_stopwords = True)
preprocessed_corpus

<ipython-input-7-f8a15d8180b8>:11: FutureWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  cleaned_corpus = pd.Series()
<ipython-input-7-f8a15d8180b8>:20: FutureWarning: The series.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  cleaned_corpus = cleaned_corpus.append(pd.Series(' '.join(qs)))
<ipython-input-7-f8a15d8180b8>:20: FutureWarning: The series.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  cleaned_corpus = cleaned_corpus.append(pd.Series(' '.join(qs)))
<ipython-input-7-f8a15d8180b8>:20: FutureWarning: The series.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  cleaned_corpus = cleaned_corpus.append(pd.Series(' '.join(qs)))


['read natural language process',
 'natural language process make computers comprehend language data',
 'field natural language process evolve everyday']

Building the vocabulary


In [14]:
set_of_words = set()
for sentence in preprocessed_corpus:
    for word in sentence.split():
        set_of_words.add(word)
vocab = list(set_of_words)
print(vocab)

['read', 'process', 'language', 'natural', 'field', 'evolve', 'make', 'computers', 'comprehend', 'everyday', 'data']


In [15]:
position = {}
for i, token in enumerate(vocab):
    position[token] = i
print(position)

{'read': 0, 'process': 1, 'language': 2, 'natural': 3, 'field': 4, 'evolve': 5, 'make': 6, 'computers': 7, 'comprehend': 8, 'everyday': 9, 'data': 10}


Creating a bow matrix

In [16]:
bow_matrix = np.zeros((len(preprocessed_corpus), len(vocab)))

In [17]:
for i, preprocessed_sentence in enumerate(preprocessed_corpus):
    for token in preprocessed_sentence.split():
        bow_matrix[i][position[token]] = bow_matrix[i][position[token]] + 1

In [18]:
bow_matrix

array([[1., 1., 1., 1., 0., 0., 0., 0., 0., 0., 0.],
       [0., 1., 2., 1., 0., 0., 1., 1., 1., 0., 1.],
       [0., 1., 1., 1., 1., 1., 0., 0., 0., 1., 0.]])

Using Countvectorizer for all the above process

In [19]:
vectorizer=CountVectorizer()
bow_matrix=vectorizer.fit_transform(preprocessed_corpus)

In [20]:
bow_matrix

<3x11 sparse matrix of type '<class 'numpy.int64'>'
	with 17 stored elements in Compressed Sparse Row format>

In [21]:
print(vectorizer.get_feature_names_out())
bow_matrix.toarray()


['comprehend' 'computers' 'data' 'everyday' 'evolve' 'field' 'language'
 'make' 'natural' 'process' 'read']


array([[0, 0, 0, 0, 0, 0, 1, 0, 1, 1, 1],
       [1, 1, 1, 0, 0, 0, 2, 1, 1, 1, 0],
       [0, 0, 0, 1, 1, 1, 1, 0, 1, 1, 0]])

In [22]:
vectorizer_ngram_range = CountVectorizer(analyzer='word',ngram_range=(1,3))
bow_matrix_ngram =vectorizer_ngram_range.fit_transform(preprocessed_corpus)
print(vectorizer_ngram_range.get_feature_names_out())
print(bow_matrix_ngram.toarray())

['comprehend' 'comprehend language' 'comprehend language data' 'computers'
 'computers comprehend' 'computers comprehend language' 'data' 'everyday'
 'evolve' 'evolve everyday' 'field' 'field natural'
 'field natural language' 'language' 'language data' 'language process'
 'language process evolve' 'language process make' 'make' 'make computers'
 'make computers comprehend' 'natural' 'natural language'
 'natural language process' 'process' 'process evolve'
 'process evolve everyday' 'process make' 'process make computers' 'read'
 'read natural' 'read natural language']
[[0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 1 0 0 0 0 0 1 1 1 1 0 0 0 0 1 1 1]
 [1 1 1 1 1 1 1 0 0 0 0 0 0 2 1 1 0 1 1 1 1 1 1 1 1 0 0 1 1 0 0 0]
 [0 0 0 0 0 0 0 1 1 1 1 1 1 1 0 1 1 0 0 0 0 1 1 1 1 1 1 0 0 0 0 0]]


Max features in countvectorizer helps us to keep a cap on the number of features that can be used. Keep in mind that increasing the number of dimension can lead to overfitting which is also known as the curse of dimensionality

In [23]:
vectorizer_ngram_range = CountVectorizer(analyzer='word',ngram_range=(1,3),max_features=6)
bow_matrix_ngram =vectorizer_ngram_range.fit_transform(preprocessed_corpus)
print(vectorizer_ngram_range.get_feature_names_out())
print(bow_matrix_ngram.toarray())

['language' 'language process' 'natural' 'natural language'
 'natural language process' 'process']
[[1 1 1 1 1 1]
 [2 1 1 1 1 1]
 [1 1 1 1 1 1]]


Using max_df and min_fdf which will ignore the number of words whose frequency is higher then the max_df as well has ignore the words that occurs in lesser amount then min_df


Issue with the bag of words approach


*   OOV problem
*   Cannot capture the semantic meaning behind the sentences



In [24]:
from sklearn.feature_extraction.text import TfidfVectorizer

vectorizer = TfidfVectorizer()
tf_idf_matrix = vectorizer.fit_transform(preprocessed_corpus)


In [25]:
print(vectorizer.get_feature_names_out())
print(tf_idf_matrix.toarray())
print("\nThe shape of the TF-IDF matrix is: ", tf_idf_matrix.shape)


['comprehend' 'computers' 'data' 'everyday' 'evolve' 'field' 'language'
 'make' 'natural' 'process' 'read']
[[0.         0.         0.         0.         0.         0.
  0.41285857 0.         0.41285857 0.41285857 0.69903033]
 [0.40512186 0.40512186 0.40512186 0.         0.         0.
  0.478543   0.40512186 0.2392715  0.2392715  0.        ]
 [0.         0.         0.         0.49711994 0.49711994 0.49711994
  0.29360705 0.         0.29360705 0.29360705 0.        ]]

The shape of the TF-IDF matrix is:  (3, 11)


Above code demonstrate the affect using L1 norm and L2 norm. The sum of all the mean absolute value of the vectors for that particular document should be 1 likewise the sum of all the means square value of the vector for the document should be 1 in L2 norm. By default tfidf uses L2 norm. While there is no difference in the vectorization process the value assigned to those tokens would be different for those two methods

While tf-idf is computationally fast and provides a way to assign weights to the words that are few amount of times it still fails to address following issues:


*   Slow for large vocabulary
*   cannot represent the semantics meaning of the sentences



**Cosine similarity**
If two words have similar meaning then their magnitude and direction shoulde be similar. We can calculate consine similarity which gives values from -1 to +1
+1 indicates that the varibales are prefectly similar and -1 indicates they are completely opposite.

In [26]:
def cosine_similarity(vector1, vector2):
  vector1=np.array(vector1)
  vector2=np.array(vector2)
  return np.dot(vector1, vector2) / (np.sqrt(np.sum(vector1**2)) * np.sqrt(np.sum(vector2**2)))


In [28]:
vectorizer = CountVectorizer()
bow_matrix = vectorizer.fit_transform(preprocessed_corpus)

In [31]:
print(vectorizer.get_feature_names_out())
print(bow_matrix.toarray())

['comprehend' 'computers' 'data' 'everyday' 'evolve' 'field' 'language'
 'make' 'natural' 'process' 'read']
[[0 0 0 0 0 0 1 0 1 1 1]
 [1 1 1 0 0 0 2 1 1 1 0]
 [0 0 0 1 1 1 1 0 1 1 0]]


In [32]:
for i in range(bow_matrix.shape[0]):
    for j in range(i + 1, bow_matrix.shape[0]):
        print("The cosine similarity between the documents ", i, "and", j, "is: ",
              cosine_similarity(bow_matrix.toarray()[i], bow_matrix.toarray()[j]))

The cosine similarity between the documents  0 and 1 is:  0.6324555320336759
The cosine similarity between the documents  0 and 2 is:  0.6123724356957946
The cosine similarity between the documents  1 and 2 is:  0.5163977794943223


One hot vectors
Represents categorical values into numeric values. Assigns 1 to the position of the token and zero elsewherer

In [35]:
sentence = ["We are reading about Natural Language Processing Here"]
corpus2=pd.Series(sentence)
corpus2

0    We are reading about Natural Language Processi...
dtype: object

In [36]:
# Preprocessing with Lemmatization here
preprocessed_corpus = preprocess(corpus, keep_list = [], stemming =False, stem_type = None,lemmatization = True, remove_stopwords =True)
preprocessed_corpus


<ipython-input-7-f8a15d8180b8>:11: FutureWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  cleaned_corpus = pd.Series()
<ipython-input-7-f8a15d8180b8>:20: FutureWarning: The series.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  cleaned_corpus = cleaned_corpus.append(pd.Series(' '.join(qs)))


['read natural language process']

In [40]:
set_of_words = set()
for word in preprocessed_corpus[0].split():
    set_of_words.add(word)
vocab = list(set_of_words)
print(vocab)

['read', 'language', 'process', 'natural']


In [44]:
preprocessed_corpus

['read natural language process']

In [45]:
preprocessed_corpus[0]

'read natural language process'

In [53]:
position = {}
for i, token in enumerate(vocab):
    position[token] = i
print(position)

{'read': 0, 'language': 1, 'process': 2, 'natural': 3}


In [54]:
one_hot_matrix = np.zeros((len(preprocessed_corpus[0].split()), len(vocab)))
one_hot_matrix.shape

(4, 4)

In [55]:
one_hot_matrix

array([[0., 0., 0., 0.],
       [0., 0., 0., 0.],
       [0., 0., 0., 0.],
       [0., 0., 0., 0.]])

In [57]:
for i, token in enumerate(preprocessed_corpus[0].split()):
    one_hot_matrix[i][position[token]] = 1
    print(position[token])


0
3
1
2


In [58]:
one_hot_matrix

array([[1., 0., 0., 0.],
       [0., 0., 0., 1.],
       [0., 1., 0., 0.],
       [0., 0., 1., 0.]])